# Food Delivery Time estimation

## 1. Data Preparation

### 1.1 Import libraries

In [1]:
from module.distance_calculator import DistanceCalculator

import numpy as np
import pandas as pd

from math import sqrt, cos

import networkx as nx

import datetime
from datetime import time, date, datetime

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn import metrics


from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate

ModuleNotFoundError: No module named 'seaborn'

### 1.2 Exploratory Data Analysis (EDA)

### 1.2.1 Examine data

In [ ]:
# Read in the data
df = pd.read_excel("Capstone SampleData 14days.xlsx", sheet_name="Sheet1", header=0)

In [ ]:
# Check the data types
df.head()

In [ ]:
# Check the data informations
df.info()

In [ ]:
# Check the data shape
df.shape

### 1.2.2 Remove unused columns

In [ ]:
# Remove the columns that are not needed
drop_list = ["calledMerchantTime",
             "arrivedAtMerchantTime",
             "foodDeliveredTime",
             "riderInitial.lat",
             "riderInitial.long",
             "MerchantName",
             "NationFoodCategory",
             "FoodCategories"]
try:
    df.drop(drop_list, axis=1, inplace=True)
except KeyError:
    print("Columns already removed")

display(df.head(3))

### 1.2.3 Change data types (time, date -> datetime)
datetime data type can find interval but both time and date data types cannot.

In [ ]:
def to_datetime(df, date_col, time_col):
    return pd.to_datetime(df[date_col].astype(str) + " " + df[time_col].astype(str))

In [ ]:
if not isinstance(df["mealPickedUpTime"].iloc[0], datetime):
    df["mealPickedUpTime"]    = to_datetime(df, "JobAcceptedDate", "mealPickedUpTime")
if not isinstance(df["arrivedAtCustLocationTime"].iloc[0], datetime):
    df["arrivedAtCustLocationTime"] = to_datetime(df, "JobAcceptedDate", "arrivedAtCustLocationTime")

### 1.2.4 Create target from existed columns

In [ ]:
df["DeliveryTime"] = df["arrivedAtCustLocationTime"] - df["mealPickedUpTime"]
df["DeliveryTime (s)"] = df["DeliveryTime"].apply(lambda x: x.total_seconds())
df["DeliveryTime (m)"] = df["DeliveryTime"].apply(lambda x: round(x.total_seconds()/60))

In [ ]:
# Remove the columns that are not needed
drop_list = ["arrivedAtCustLocationTime",
             "mealPickedUpTime",
             "DeliveryTime"]
try:
    df.drop(drop_list, axis=1, inplace=True)
except KeyError:
    print("Columns already removed")

display(df.head(3))

In [ ]:
df.describe()

In [ ]:
df["DeliveryTime (s)"].hist(bins=100)
plt.title("Delivery Time Distribution (s)")
plt.xlabel("Delivery Time (s)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
df["DeliveryTime (m)"].hist(bins=100)
plt.title("Delivery Time Distribution (m)")
plt.xlabel("Delivery Time (m)")
plt.ylabel("Frequency")
plt.show()

### 1.2.5 Add additional features

In [ ]:
def get_euc(coords_1, coords_2):
    R = 6371000; conversion_const = 0.0174533
    c_1 = (coords_1[0]*conversion_const, coords_1[1]*conversion_const)
    c_2 = (coords_2[0]*conversion_const, coords_2[1]*conversion_const)
    delta_phi = abs(c_1[1]-c_2[1])
    theta = c_1[0]
    delta_theta = abs(c_1[0]-c_2[0])
    del_x = R*cos(theta)*delta_phi 
    del_y = R*delta_theta
    return sqrt(del_x**2 + del_y**2)

In [ ]:
df["EucDist"] = df.apply(lambda x: get_euc((x["Merchant.Lat"],x["Merchant.Lng"]),(x["Customer.lat"],x["Customer.lng"])), axis=1)

In [ ]:
mapper = {0:"Mon", 1:"Tue", 2:"Wed", 3:"Thu", 4:"Fri", 5:"Sat", 6:"Sun"} # TODO
df["day_of_week"] = df["JobAcceptedDate"].apply(lambda x: x.weekday())
df["day_of_week_name"] = df["JobAcceptedDate"].apply(lambda x: mapper[x.weekday()])

df["isHoliday"] = df["JobAcceptedDate"].apply(lambda x: int(x == pd.Timestamp('2020-10-13 00:00:00')))
df["isHoliday"] = ( df["isHoliday"] | ( (df["day_of_week"] == 5) | (df["day_of_week"] == 6) ) ).astype(int)

In [ ]:
df.head()

In [ ]:
# One Hot encode
nominal_columns = ["day_of_week_name"]
dummy_df = pd.get_dummies(df[nominal_columns], drop_first=False)
df = pd.concat([df, dummy_df], axis=1)
df = df.drop(nominal_columns, axis=1)

In [ ]:
df.head()

In [ ]:
## Try to convert one hot to angular distance (https://www.mikulskibartosz.name/time-in-machine-learning/)
df["day_of_week_sin"] = np.sin(df["day_of_week"]*(2.*np.pi/7))
df["day_of_week_cos"] = np.cos(df["day_of_week"]*(2.*np.pi/7))

In [ ]:
df.head()

In [ ]:
D = DistanceCalculator()

In [ ]:
nx.is_strongly_connected(D.G)

In [ ]:
# tmp = df.iloc[:10].copy()
# tmp["Astar"] = tmp.apply(lambda x: D.shortestDistance((x["Merchant.Lat"], x["Merchant.Lng"]), (x["Customer.lat"], x["Customer.lng"])), axis=1)

In [ ]:
# tmp[["EucDist", "Astar"]]

In [ ]:
df["ShortestDist"] = 0

In [ ]:
df[["EucDist", "ShortestDist"]]

In [ ]:
# df.apply(lambda x: D.shortestDistance((x["Merchant.Lat"], x["Merchant.Lng"]), (x["Customer.lat"], x["Customer.lng"])), axis=1)

In [ ]:
batch_size = 20
for i in range(0, df.shape[0], batch_size):
    print(i)
    df.loc[i:i+batch_size, "ShortestDist"] = df.loc[i:i+batch_size].apply(lambda x: D.shortestDistance((x["Merchant.Lat"], x["Merchant.Lng"]), (x["Customer.lat"], x["Customer.lng"])), axis=1)
    df.to_csv("Sample 14days (cleaned)", index=False)

In [ ]:
df[df["ShortestDist"] == 0]

In [ ]:
df[["EucDist", "ShortestDist"]]

In [ ]:
# idx = 2
# u = df[["Merchant.Lat", "Merchant.Lng"]].iloc[idx].values
# v = df[["Customer.lat", "Customer.lng"]].iloc[idx].values

In [ ]:
# D.shortestDistance(u, v)

### 1.2.6 Extract input/output for train model

In [ ]:
# LOAD CLEANED DATASET
df = pd.read_csv("Sample 14days (cleaned)")

In [ ]:
df[df["ShortestDist"]==0]

In [ ]:
df.loc[df["ShortestDist"]==0, "ShortestDist"] = df.loc[df["ShortestDist"]==0, "EucDist"]

In [ ]:
df[df["ShortestDist"]==0]

In [ ]:
y_s = df.pop("DeliveryTime (s)")
y_m = df.pop("DeliveryTime (m)")
# # data without day of week
# selected_cols_1 = ["Merchant.Lat", "Merchant.Lng", "Customer.lat", "Customer.lng", "EucDist"]
# # data with day of week (onehot)
# selected_cols_2 = ["Merchant.Lat", "Merchant.Lng", "Customer.lat", "Customer.lng", "EucDist", "isHoliday",\
#     'day_of_week_name_Mon', 'day_of_week_name_Tue', 'day_of_week_name_Wed', 'day_of_week_name_Thu', 'day_of_week_name_Fri', 'day_of_week_name_Sat', 'day_of_week_name_Sun']
# # data with day of week (angular distance)
# selected_cols_3 = ["Merchant.Lat", "Merchant.Lng", "Customer.lat", "Customer.lng", "EucDist", "isHoliday", "day_of_week_sin",	"day_of_week_cos"]
selected_cols_1 = ["Merchant.Lat", "Merchant.Lng", "Customer.lat", "Customer.lng", "EucDist"]
selected_cols_2 = ["Merchant.Lat", "Merchant.Lng", "Customer.lat", "Customer.lng", "EucDist", "ShortestDist"]

X = [df[selected_cols_1].copy(), df[selected_cols_2].copy()]

## 3. Model Regression

### 3.1 Split data for train and test

In [ ]:
Xs_trains = []; Xs_tests = []; ys_trains = []; ys_tests = []
Xm_trains = []; Xm_tests = []; ym_trains = []; ym_tests = []

for i in range(len(X)):
    Xs_train, Xs_test, ys_train, ys_test = train_test_split(X[i], y_s, test_size=0.20, random_state=0)
    Xm_train, Xm_test, ym_train, ym_test = train_test_split(X[i], y_m, test_size=0.20, random_state=0)
    Xs_trains.append(Xs_train); Xs_tests.append(Xs_test); ys_trains.append(ys_train); ys_tests.append(ys_test)
    Xm_trains.append(Xm_train); Xm_tests.append(Xm_test); ym_trains.append(ym_train); ym_tests.append(ym_test)

### 3.2 Model Metrics

In [ ]:
met = pd.DataFrame({"Model": [], "MAE": [], "MSE": [], "RMSE": [], "R2": []})
model_metrics_s = [met.copy(), met.copy()]
model_metrics_m = [met.copy(), met.copy()]

### 3.3 Baseline model (Average)

In [ ]:
all_predictions_s = [[],[]]; all_predictions_m = [[],[]]

### 3.3.1 As second

In [ ]:
avg_predictions_s = np.full(len(ys_tests[i]), np.mean(ys_trains[i]))
for i in range(len(X)):
    MAE  = metrics.mean_absolute_error(ys_tests[i], avg_predictions_s)
    MSE  = metrics.mean_squared_error(ys_tests[i], avg_predictions_s)
    R2   = metrics.r2_score(ys_tests[i], avg_predictions_s)
    RMSE = np.sqrt(metrics.mean_squared_error(ys_test, avg_predictions_s))

    all_predictions_s[i].append(avg_predictions_s)
    model_metrics_s[i].loc[len(model_metrics_s[i])] = list(["Mean ", MAE, MSE, RMSE, R2])

### 3.3.2 As minute

In [ ]:
avg_predictions_m =  np.full(len(ym_tests[i]), np.mean(ym_trains[i]))
for i in range(len(X)):
    MAE  = metrics.mean_absolute_error(ym_test, avg_predictions_m)
    MSE  = metrics.mean_squared_error(ym_test, avg_predictions_m)
    R2   = metrics.r2_score(ym_test, avg_predictions_m)
    RMSE = np.sqrt(metrics.mean_squared_error(ym_test, avg_predictions_m))

    all_predictions_m[i].append(avg_predictions_m)
    model_metrics_m[i].loc[len(model_metrics_m[i])] = list(["Mean ", MAE, MSE, RMSE, R2])

### 3.3 Linear Regression

#### 3.3.1 As second

In [ ]:
for i in range(len(X)):
    lr = LinearRegression()
    lr.fit(Xs_trains[i], ys_trains[i])
    lr_predictions_s = lr.predict(Xs_tests[i])

    MAE  = metrics.mean_absolute_error(ys_tests[i], lr_predictions_s)
    MSE  = metrics.mean_squared_error(ys_tests[i], lr_predictions_s)
    R2   = metrics.r2_score(ys_tests[i], lr_predictions_s)
    RMSE = np.sqrt(metrics.mean_squared_error(ys_tests[i], lr_predictions_s))

    all_predictions_s[i].append(lr_predictions_s)
    model_metrics_s[i].loc[len(model_metrics_s[i])] = list(["Linear Regression", MAE, MSE, RMSE, R2])

#### 3.3.2 As minute

In [ ]:
for i in range(len(X)):
    lr = LinearRegression()
    lr.fit(Xm_trains[i], ym_trains[i])
    lr_predictions_m = lr.predict(Xm_tests[i])
    lr_predictions_m = np.round(lr_predictions_m)

    MAE  = metrics.mean_absolute_error(ym_tests[i], lr_predictions_m)
    MSE  = metrics.mean_squared_error(ym_tests[i], lr_predictions_m)
    R2   = metrics.r2_score(ym_tests[i], lr_predictions_m)
    RMSE = np.sqrt(metrics.mean_squared_error(ym_tests[i], lr_predictions_m))

    all_predictions_m[i].append(lr_predictions_m)
    model_metrics_m[i].loc[len(model_metrics_m[i])] = list(["Linear Regression", MAE, MSE, RMSE, R2])

### 3.4 Random Forest

#### 3.4.1 As second

In [ ]:
for i in range(len(X)):
    rf = RandomForestRegressor()
    rf.fit(Xs_trains[i], ys_trains[i])
    rf_predictions_s = rf.predict(Xs_tests[i])
    
    MAE  = metrics.mean_absolute_error(ys_tests[i], rf_predictions_s)
    MSE  = metrics.mean_squared_error(ys_tests[i], rf_predictions_s)
    R2   = metrics.r2_score(ys_tests[i], rf_predictions_s)
    RMSE = np.sqrt(metrics.mean_squared_error(ys_tests[i], rf_predictions_s))

    all_predictions_s[i].append(rf_predictions_s)
    model_metrics_s[i].loc[len(model_metrics_s[i])] = list(["Random Forest", MAE, MSE, RMSE, R2])

#### 3.4.2 As minute

In [ ]:
for i in range(len(X)):
    rf = RandomForestRegressor()
    rf.fit(Xm_trains[i], ym_trains[i])
    rf_predictions_m = rf.predict(Xm_tests[i])
    rf_predictions_m = np.round(rf_predictions_m)

    MAE  = metrics.mean_absolute_error(ym_tests[i], rf_predictions_m)
    MSE  = metrics.mean_squared_error(ym_tests[i], rf_predictions_m)
    R2   = metrics.r2_score(ym_tests[i], rf_predictions_m)
    RMSE = np.sqrt(metrics.mean_squared_error(ym_tests[i], rf_predictions_m))

    all_predictions_m[i].append(rf_predictions_m)
    model_metrics_m[i].loc[len(model_metrics_m[i])] = list(["Random Forest", MAE, MSE, RMSE, R2])

### 3.5 Gradient Boosted Desicion Tree

#### 3.5.1 As second

In [ ]:
for i in range(len(X)):
    gbdt = GradientBoostingRegressor(n_estimators=1000, max_depth=7, random_state=0)
    gbdt.fit(Xs_trains[i], ys_trains[i])
    gbdt_predictions_s = gbdt.predict(Xs_tests[i])
    
    MAE  = metrics.mean_absolute_error(ys_tests[i], gbdt_predictions_s)
    MSE  = metrics.mean_squared_error(ys_tests[i], gbdt_predictions_s)
    R2   = metrics.r2_score(ys_tests[i], gbdt_predictions_s)
    RMSE = np.sqrt(metrics.mean_squared_error(ys_tests[i], gbdt_predictions_s))

    all_predictions_s[i].append(gbdt_predictions_s)
    model_metrics_s[i].loc[len(model_metrics_s[i])] = list(["Gradient Boosted Decision Tree", MAE, MSE, RMSE, R2])

#### 3.5.2 As minute

In [ ]:
for i in range(len(X)):
    gbdt = GradientBoostingRegressor(n_estimators=1000, max_depth=7, random_state=0)
    gbdt.fit(Xm_trains[i], ym_trains[i])
    gbdt_predictions_m = gbdt.predict(Xm_tests[i])
    gbdt_predictions_m = np.round(gbdt_predictions_m)

    MAE  = metrics.mean_absolute_error(ym_tests[i], gbdt_predictions_m)
    MSE  = metrics.mean_squared_error(ym_tests[i], gbdt_predictions_m)
    R2   = metrics.r2_score(ym_tests[i], gbdt_predictions_m)
    RMSE = np.sqrt(metrics.mean_squared_error(ym_tests[i], gbdt_predictions_m))

    all_predictions_m[i].append(gbdt_predictions_m)
    model_metrics_m[i].loc[len(model_metrics_m[i])] = list(["Gradient Boosted Decision Tree", MAE, MSE, RMSE, R2])

## 4. Model Evaluation

In [ ]:
# DataFeats = ["Without day_of_week", "With day_of_week OneHot", "With day_of_week Angular"]
# table_names = ["Metrics without day_of_week as second", "Metrics with day_of_week_OneHot as second", "Metrics with day_of_week Angular as second"]
# for i in range(len(X)):
#     print(DataFeats[i])
#     display(model_metrics_s[i])
#     #model_metrics_s[i].to_csv(table_names[i] + ".csv", index=False)

In [ ]:
DataFeats = ["Without ShortestPath", "With ShortestPath"]
table_names = ["Metrics without ShortestPath as second", "Metrics with ShortestPath as second"]
for i in range(len(X)):
    print(DataFeats[i])
    display(model_metrics_s[i])
    model_metrics_s[i].to_csv(table_names[i] + ".csv", index=False)

In [ ]:
# table_names = ["Metrics without day_of_week as minute", "Metrics with day_of_week_OneHot as minute", "Metrics with day_of_week Angular as minute"]
# for i in range(len(X)):
#     print(DataFeats[i])
#     display(model_metrics_m[i])
#     #model_metrics_m[i].to_csv(table_names[i] + ".csv", index=False)

In [ ]:
table_names = ["Metrics without ShortestPath as minute", "Metrics with ShortestPath as minute"]
for i in range(len(X)):
    print(DataFeats[i])
    display(model_metrics_m[i])
    model_metrics_m[i].to_csv(table_names[i] + ".csv", index=False)

# Knowledge

In [ ]:
plt.scatter(df["EucDist"], y_s, alpha=0.1)
plt.title("Euclidian distance (m) - Delivery time (s)")
plt.xlabel("Euclidian distance (m)")
plt.ylabel("Delivery time (s)")
plt.show()

In [ ]:
plt.scatter(df["EucDist"], y_m, alpha=0.1)
plt.title("Euclidian distance (m) - Delivery time (m)")
plt.xlabel("Euclidian distance (m)")
plt.ylabel("Delivery time (m)")
plt.show()

In [ ]:
plt.scatter(df["ShortestDist"], y_s, alpha=0.1)
plt.title("Shortest Path distance (m) - Delivery time (s)")
plt.xlabel("Shortest Path distance (m)")
plt.ylabel("Delivery time (s)")
plt.show()

In [ ]:
plt.scatter(df["ShortestDist"], y_m, alpha=0.1)
plt.title("Shortest Path distance (m) - Delivery time (s)")
plt.xlabel("Shortest Path distance (m)")
plt.ylabel("Delivery time (s)")
plt.show()

In [ ]:
df["EucDist"].hist(bins=100)
plt.plot()

In [ ]:
df["ShortestDist"].hist(bins=100)

# Model Variance

In [ ]:
Names = ["Mean", "Lr", "Rf", "GBDT"]
#DataFeats
fig, axs = plt.subplots(len(Names), len(X), figsize=(15, 15))
fig.tight_layout(pad=5.0)
for i in range(len(Names)):
    for j in range(len(X)):
        axs[i, j].scatter(all_predictions_s[j][i], ys_tests[j], alpha=0.1)
        axs[i, j].set_title(Names[i]+" "+DataFeats[j])
        axs[i, j].set_xlabel("Predicted")
        axs[i, j].set_ylabel("Actual")

In [ ]:
fig, axs = plt.subplots(len(Names), len(X), figsize=(15, 15))
fig.tight_layout(pad=5.0)
for i in range(len(Names)):
    for j in range(len(X)):
        axs[i, j].scatter(all_predictions_m[j][i], ym_tests[j],  alpha=0.1)
        axs[i, j].set_title(Names[i]+" "+DataFeats[j])
        axs[i, j].set_xlabel("Predicted")
        axs[i, j].set_ylabel("Actual")

# Error distribution

In [ ]:
fig, axs = plt.subplots(len(Names), len(X), figsize=(15, 15))
fig.tight_layout(pad=5.0)
for i in range(len(Names)):
    for j in range(len(X)):
        sns.histplot((ys_test-all_predictions_s[j][i]), bins=100, ax=axs[i, j], kde=True, stat="density", linewidth=0)
        axs[i, j].set_title(Names[i]+" "+DataFeats[j])
        axs[i, j].set_xlabel("Error")
        axs[i, j].set_ylabel("Frequency")
        axs[i, j].set_xlim(-1500, 1500)
        axs[i, j].set_ylim(0, 0.0015)

In [ ]:
fig, axs = plt.subplots(len(Names), len(X), figsize=(15, 15))
fig.tight_layout(pad=5.0)
for i in range(len(Names)):
    for j in range(len(X)):
        sns.histplot((ym_test-all_predictions_m[j][i]), bins=100, ax=axs[i, j], kde=True, stat="density", linewidth=0)
        axs[i, j].set_title(Names[i]+" "+DataFeats[j])
        axs[i, j].set_xlabel("Error")
        axs[i, j].set_ylabel("Frequency")
        axs[i, j].set_xlim(-30, 30)
        axs[i, j].set_ylim(0, 0.08)

## 6. Hyperparameter tuning

In [ ]:
from module.distance_calculator import DistanceCalculator

import numpy as np
import pandas as pd

from math import sqrt, cos

import networkx as nx

import datetime
from datetime import time, date, datetime

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn import metrics


from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate

In [ ]:
# LOAD CLEANED DATASET
df = pd.read_csv("Sample 14days (cleaned)")
df.loc[df["ShortestDist"]==0, "ShortestDist"] = df.loc[df["ShortestDist"]==0, "EucDist"]

In [ ]:
df.head(5)

In [ ]:
import pickle as pkl
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from mpl_toolkits.mplot3d import Axes3D
from pylab import cm
import matplotlib.ticker as mticker

In [ ]:
# gbdt = GradientBoostingRegressor(random_state=314159)
# # gbdt.get_params(deep=True)

# param_grid = { 
#  "n_estimators": [100, 200, 300, 500, 1000, 1500, 2000],
#  "max_features": [1.0, "sqrt", "log2"], # "auto" is deprecated, use 1.0 instead
#  "max_depth": [2, 3, 4, 5, 6, 7, 8],
# }

# CV_gbdt_s = GridSearchCV(estimator=gbdt, param_grid=param_grid)
# CV_gbdt_s.fit(Xs_train, ys_train)

In [ ]:
# pkl.dump(CV_gbdt_s, open("CV_gbdt_s.pkl", "wb"))

In [ ]:
CV_gbdt_s = pkl.load(open("CV_gbdt_s.pkl", "rb"))

In [ ]:
# gbdt = GradientBoostingRegressor(random_state=314159)
# # gbdt.get_params(deep=True)

# param_grid = { 
#  "n_estimators": [100, 200, 300, 500, 1000, 1500, 2000],
#  "max_features": [1.0, "sqrt", "log2"], # "auto" is deprecated, use 1.0 instead
#  "max_depth" : [2, 3, 4, 5, 6, 7, 8],
# }

# CV_gbdt_m = GridSearchCV(estimator=gbdt, param_grid=param_grid)
# CV_gbdt_m.fit(Xm_train, ym_train)

In [ ]:
# pkl.dump(CV_gbdt_m, open("CV_gbdt_m.pkl", "wb"))

In [ ]:
CV_gbdt_m = pkl.load(open("CV_gbdt_m.pkl", "rb"))

In [ ]:
gbdt_s = CV_gbdt_s.best_estimator_

In [ ]:
# pkl.dump(gbdt_s, open("gbdt_s.pkl", "wb"))

In [ ]:
CV_gbdt_s.best_params_

In [ ]:
CV_s_result = pd.DataFrame(CV_gbdt_s.cv_results_)

In [ ]:
CV_s_result.info()

In [ ]:
CV_s_result[["param_max_depth", "param_max_features", "param_n_estimators", "mean_test_score", "std_test_score"]].sort_values(by="mean_test_score", ascending=False)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

x = CV_s_result["param_max_depth"].values.astype("int")
y = CV_s_result["param_max_features"].map({1.0: 0, "sqrt": 1, "log2": 2}).values.astype("int")
z = CV_s_result["param_n_estimators"].values.astype("int")
colo = CV_s_result["mean_test_score"].values

color_map = cm.ScalarMappable(cmap=cm.Greens_r)
color_map.set_array(colo)

img = ax.scatter(xs=x, ys=y, zs=z, c=colo, cmap=cm.Greens_r, s=100)
cbar = fig.colorbar(img, ticks=np.linspace(0.70, 0.79, 10), orientation='vertical', shrink=0.5, aspect=10, pad=0.1)
cbar.set_label('mean_test_score', rotation=270, labelpad=20)

ax.set_title("mean R2 score of CV_gbdt_s")
ax.set_xlabel('param_max_depth')
ax.set_ylabel('param_max_features')
ax.set_zlabel('param_n_estimators')

ax.axes.yaxis.set_major_locator(mticker.FixedLocator(y))
ax.axes.yaxis.set_ticklabels(CV_s_result["param_max_features"].map({1.0: "auto", "sqrt": "sqrt", "log2": "log2"}))
plt.show()

In [ ]:
gbdt_predictions_s = gbdt_s.predict(Xs_test)

MAE  = metrics.mean_absolute_error(ys_test, gbdt_predictions_s)
MSE  = metrics.mean_squared_error(ys_test, gbdt_predictions_s)
R2   = metrics.r2_score(ys_test, gbdt_predictions_s)
RMSE = np.sqrt(metrics.mean_squared_error(ys_test, gbdt_predictions_s))

In [ ]:
MAE, MSE, R2, RMSE

In [ ]:
gbdt_predictions_s_round = np.round(gbdt_s.predict(Xs_test))

MAE  = metrics.mean_absolute_error(ys_test, gbdt_predictions_s_round)
MSE  = metrics.mean_squared_error(ys_test, gbdt_predictions_s_round)
R2   = metrics.r2_score(ys_test, gbdt_predictions_s_round)
RMSE = np.sqrt(metrics.mean_squared_error(ys_test, gbdt_predictions_s_round))

In [ ]:
MAE, MSE, R2, RMSE

In [ ]:
gbdt_s_result = pd.DataFrame({"Actual": ys_test, "Predicted_0": gbdt_predictions_s, "Predicted_1": gbdt_predictions_s_round})
gbdt_s_result

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(gbdt_predictions_s, ys_test, alpha=0.1)
axs[0].set_title("gbdt_s")
axs[0].set_xlabel("Predicted")
axs[0].set_ylabel("Actual")

axs[1].scatter(gbdt_predictions_s_round, ys_test, alpha=0.1)
axs[1].set_title("gbdt_s_rounded")
axs[1].set_xlabel("Predicted")
axs[1].set_ylabel("Actual")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(ys_test, gbdt_predictions_s, alpha=0.1)
axs[0].set_title("gbdt_s")
axs[0].set_xlabel("Actual")
axs[0].set_ylabel("Predicted")

axs[1].scatter(ys_test, gbdt_predictions_s_round, alpha=0.1)
axs[1].set_title("gbdt_s_rounded")
axs[1].set_xlabel("Actual")
axs[1].set_ylabel("Predicted")

In [ ]:

# TODO pred/act plot
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(gbdt_predictions_s, ys_test/gbdt_predictions_s, alpha=0.1)
axs[0].set_title("gbdt_s")
axs[0].set_xlabel("Predicted")
axs[0].set_ylabel("Actual/Predicted")

axs[1].scatter(gbdt_predictions_s_round, ys_test/gbdt_predictions_s_round, alpha=0.1)
axs[1].set_title("gbdt_s_rounded")
axs[1].set_xlabel("Predicted")
axs[1].set_ylabel("Actual/Predicted")

In [ ]:

# TODO pred/act plot
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(ys_test/gbdt_predictions_s, gbdt_predictions_s, alpha=0.1)
axs[0].set_title("gbdt_s")
axs[0].set_xlabel("Actual/Predicted")
axs[0].set_ylabel("Predicted")

axs[1].scatter(ys_test/gbdt_predictions_s_round, gbdt_predictions_s_round,alpha=0.1)
axs[1].set_title("gbdt_s_rounded")
axs[1].set_xlabel("Actual/Predicted")
axs[1].set_ylabel("Predicted")

In [ ]:

# TODO act, act/pred plot
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(ys_test, ys_test/gbdt_predictions_s, alpha=0.1)
axs[0].set_title("gbdt_s")
axs[0].set_xlabel("Actual")
axs[0].set_ylabel("Actual/Predicted")

axs[1].scatter(ys_test, ys_test/gbdt_predictions_s_round, alpha=0.1)
axs[1].set_title("gbdt_s_rounded")
axs[1].set_xlabel("Actual")
axs[1].set_ylabel("Actual/Predicted")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)

sns.histplot((ys_test-gbdt_predictions_s), bins=100, ax=axs[0], kde=True, stat="density", linewidth=0)
axs[0].set_title("CV_gbdt_s")
axs[0].set_xlabel("Error")
axs[0].set_ylabel("Frequency")
axs[0].set_xlim(-1500, 1500)
axs[0].set_ylim(0, 0.0015)

sns.histplot((ys_test-gbdt_predictions_s_round), bins=100, ax=axs[1], kde=True, stat="density", linewidth=0)
axs[1].set_title("CV_gbdt_s_rounded")
axs[1].set_xlabel("Error")
axs[1].set_ylabel("Frequency")
axs[1].set_xlim(-1500, 1500)
axs[1].set_ylim(0, 0.0015)

#### for m

In [ ]:
gbdt_m = CV_gbdt_m.best_estimator_

In [ ]:
gbdt_m.feature_importances_

In [ ]:
# pkl.dump(gbdt_m, open("gbdt_m.pkl", "wb"))

In [ ]:
CV_gbdt_m.best_params_

In [ ]:
pd.DataFrame(CV_gbdt_m.cv_results_).head(5)

In [ ]:
CV_m_result = pd.DataFrame(CV_gbdt_m.cv_results_)

In [ ]:
CV_m_result[["param_max_depth", "param_max_features", "param_n_estimators", "mean_test_score", "std_test_score"]].sort_values(by="mean_test_score", ascending=False)

In [ ]:
gbdt_predictions_m = gbdt_m.predict(Xm_test)

MAE  = metrics.mean_absolute_error(ym_test, gbdt_predictions_m)
MSE  = metrics.mean_squared_error(ym_test, gbdt_predictions_m)
R2   = metrics.r2_score(ym_test, gbdt_predictions_m)
RMSE = np.sqrt(metrics.mean_squared_error(ym_test, gbdt_predictions_m))

In [ ]:
MAE, MSE, R2, RMSE

In [ ]:
gbdt_predictions_m_round = np.round(gbdt_m.predict(Xm_test))

MAE  = metrics.mean_absolute_error(ym_test, gbdt_predictions_m_round)
MSE  = metrics.mean_squared_error(ym_test, gbdt_predictions_m_round)
R2   = metrics.r2_score(ym_test, gbdt_predictions_m_round)
RMSE = np.sqrt(metrics.mean_squared_error(ym_test, gbdt_predictions_m_round))

In [ ]:
MAE, MSE, R2, RMSE

In [ ]:
gbdt_m_result = pd.DataFrame({"Actual": ym_test, "Predicted_0": gbdt_predictions_m, "Predicted_1": gbdt_predictions_m_round})
gbdt_m_result

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(gbdt_predictions_m, ym_test, alpha=0.1)
axs[0].set_title("gbdt_m")
axs[0].set_xlabel("Predicted")
axs[0].set_ylabel("Actual")

axs[1].scatter(gbdt_predictions_m_round, ym_test, alpha=0.1)
axs[1].set_title("gbdt_m_rounded")
axs[1].set_xlabel("Predicted")
axs[1].set_ylabel("Actual")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(ym_test, gbdt_predictions_m, alpha=0.1)
axs[0].set_title("gbdt_m")
axs[0].set_xlabel("Actual")
axs[0].set_ylabel("Predicted")

axs[1].scatter(ym_test,gbdt_predictions_m_round, alpha=0.1)
axs[1].set_title("gbdt_m_rounded")
axs[1].set_xlabel("Actual")
axs[1].set_ylabel("Predicted")

In [ ]:

# TODO pred/act plot
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(gbdt_predictions_m, ym_test/gbdt_predictions_m, alpha=0.1)
axs[0].set_title("gbdt_m")
axs[0].set_xlabel("Predicted")
axs[0].set_ylabel("Actual/Predicted")

axs[1].scatter(gbdt_predictions_m_round, ym_test/gbdt_predictions_m_round, alpha=0.1)
axs[1].set_title("gbdt_m_rounded")
axs[1].set_xlabel("Predicted")
axs[1].set_ylabel("Actual/Predicted")

In [ ]:

# TODO pred/act plot
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(ym_test/gbdt_predictions_m, gbdt_predictions_m, alpha=0.1)
axs[0].set_title("gbdt_m")
axs[0].set_xlabel("Actual/Predicted")
axs[0].set_ylabel("Predicted")

axs[1].scatter(ym_test/gbdt_predictions_m_round, gbdt_predictions_m_round, alpha=0.1)
axs[1].set_title("gbdt_m_rounded")
axs[1].set_xlabel("Actual/Predicted")
axs[1].set_ylabel("Predicted")

In [ ]:

# TODO act, act/pred plot
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(ym_test, ym_test/gbdt_predictions_m, alpha=0.1)
axs[0].set_title("gbdt_m")
axs[0].set_xlabel("Actual")
axs[0].set_ylabel("Actual/Predicted")

axs[1].scatter(ym_test, ym_test/gbdt_predictions_m_round, alpha=0.1)
axs[1].set_title("gbdt_m_rounded")
axs[1].set_xlabel("Actual")
axs[1].set_ylabel("Actual/Predicted")

## compare gbdt s,m

In [ ]:

# TODO pred/act plot s/m
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(gbdt_predictions_s, ys_test/gbdt_predictions_s, alpha=0.1)
axs[0].set_title("gbdt_s")
axs[0].set_xlabel("Predicted")
axs[0].set_ylabel("Actual/Predicted")
axs[0].set_ylim(0.35, 1.5)

axs[1].scatter(gbdt_predictions_m, ym_test/gbdt_predictions_m, alpha=0.1)
axs[1].set_title("gbdt_m")
axs[1].set_xlabel("Predicted")
axs[1].set_ylabel("Actual/Predicted")
axs[1].set_ylim(0.35, 1.5)

In [ ]:

# TODO pred/act plot s/m rounded
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(gbdt_predictions_s_round, ys_test/gbdt_predictions_s_round, alpha=0.1)
axs[0].set_title("gbdt_s_rounded")
axs[0].set_xlabel("Predicted")
axs[0].set_ylabel("Actual/Predicted")
axs[0].set_ylim(0.35, 1.5)

axs[1].scatter(gbdt_predictions_m_round, ym_test/gbdt_predictions_m_round, alpha=0.1)
axs[1].set_title("gbdt_m_rounded")
axs[1].set_xlabel("Predicted")
axs[1].set_ylabel("Actual/Predicted")
axs[1].set_ylim(0.35, 1.5)

In [ ]:

# TODO act, act/pred plot s/m
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(ys_test, ys_test/gbdt_predictions_s, alpha=0.1)
axs[0].set_title("gbdt_s")
axs[0].set_xlabel("Actual")
axs[0].set_ylabel("Actual/Predicted")
axs[0].set_ylim(0.35, 1.5)

axs[1].scatter(ym_test, ym_test/gbdt_predictions_m, alpha=0.1)
axs[1].set_title("gbdt_m")
axs[1].set_xlabel("Actual")
axs[1].set_ylabel("Actual/Predicted")
axs[1].set_ylim(0.35, 1.5)

In [ ]:

# TODO pred/act plot s/m rounded
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(ys_test, ys_test/gbdt_predictions_s_round, alpha=0.1)
axs[0].set_title("gbdt_s_rounded")
axs[0].set_xlabel("Actual")
axs[0].set_ylabel("Actual/Predicted")
axs[0].set_ylim(0.35, 1.5)

axs[1].scatter(ym_test, ym_test/gbdt_predictions_m_round, alpha=0.1)
axs[1].set_title("gbdt_m_rounded")
axs[1].set_xlabel("Actual")
axs[1].set_ylabel("Actual/Predicted")
axs[1].set_ylim(0.35, 1.5)

In [ ]:

# TODO pred/act plot s/m
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(ys_test/gbdt_predictions_s, gbdt_predictions_s, alpha=0.1)
axs[0].set_title("gbdt_s")
axs[0].set_xlabel("Actual/Predicted")
axs[0].set_ylabel("Predicted")
axs[0].set_xlim(0.35, 1.5)

axs[1].scatter(ym_test/gbdt_predictions_m, gbdt_predictions_m, alpha=0.1)
axs[1].set_title("gbdt_m")
axs[1].set_xlabel("Actual/Predicted")
axs[1].set_ylabel("Predicted")
axs[1].set_xlim(0.35, 1.5)

In [ ]:

# TODO pred/act plot s/m rounded
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)
axs[0].scatter(ys_test/gbdt_predictions_s_round, gbdt_predictions_s_round, alpha=0.1)
axs[0].set_title("gbdt_s_rounded")
axs[0].set_xlabel("Actual/Predicted")
axs[0].set_ylabel("Predicted")
axs[0].set_xlim(0.35, 1.5)

axs[1].scatter(ym_test/gbdt_predictions_m_round, gbdt_predictions_m_round, alpha=0.1)
axs[1].set_title("gbdt_m_rounded")
axs[1].set_xlabel("Actual/Predicted")
axs[1].set_ylabel("Predicted")
axs[1].set_xlim(0.35, 1.5)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
fig.tight_layout(pad=5.0)

sns.histplot((ym_test-gbdt_predictions_m), bins=100, ax=axs[0], kde=True, stat="density", linewidth=0)
axs[0].set_title("CV_gbdt_m")
axs[0].set_xlabel("Error")
axs[0].set_ylabel("Frequency")
axs[0].set_xlim(-30, 30)
axs[0].set_ylim(0, 0.08)

sns.histplot((ym_test-gbdt_predictions_m_round), bins=100, ax=axs[1], kde=True, stat="density", linewidth=0)
axs[1].set_title("CV_gbdt_m_rounded")
axs[1].set_xlabel("Error")
axs[1].set_ylabel("Frequency")
axs[1].set_xlim(-30, 30)
axs[1].set_ylim(0, 0.08)

In [ ]:
gbdt_m_result["AE_0"] = np.abs(gbdt_m_result["Actual"] - gbdt_m_result["Predicted_0"])
gbdt_m_result["AE_1"] = np.abs(gbdt_m_result["Actual"] - gbdt_m_result["Predicted_1"])
gbdt_m_result.head(1)

In [ ]:
gbdt_m_result

In [ ]:
# TODO: try to group AE_0 to cateorical
gbdt_m_pareto_0 = gbdt_m_result.value_counts("AE_0").sort_index().to_frame().reset_index()
# gbdt_m_pareto_1.index = gbdt_m_pareto_1["AE_1"].values.astype(str)
# gbdt_m_pareto_1["AE_1"] = gbdt_m_pareto_1[0]
# gbdt_m_pareto_1.drop(columns=[0], inplace=True)
# gbdt_m_pareto_1["cumpercentage"] = gbdt_m_pareto_1["AE_1"].cumsum()/gbdt_m_pareto_1["AE_1"].sum()*100
# gbdt_m_pareto_1.head(3)

gbdt_m_pareto_0.value_counts()

In [ ]:
# for AE_1 it is integer
from matplotlib.ticker import PercentFormatter
gbdt_m_pareto_1 = gbdt_m_result.value_counts("AE_1").sort_index().to_frame().reset_index()
gbdt_m_pareto_1.index = gbdt_m_pareto_1["AE_1"].values.astype(str)
gbdt_m_pareto_1["AE_1"] = gbdt_m_pareto_1[0]
gbdt_m_pareto_1.drop(columns=[0], inplace=True)
gbdt_m_pareto_1["cumpercentage"] = gbdt_m_pareto_1["AE_1"].cumsum()/gbdt_m_pareto_1["AE_1"].sum()*100
gbdt_m_pareto_1.head(3)

In [ ]:
gbdt_m_pareto_1

In [ ]:
129/2000*100

In [ ]:
# TODO pareto chart of AE_1
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(gbdt_m_pareto_1.index, gbdt_m_pareto_1["AE_1"], color="C0")
ax2 = ax.twinx()
ax2.plot(gbdt_m_pareto_1.index, gbdt_m_pareto_1["cumpercentage"], color="C1", marker="D", ms=7)
ax2.yaxis.set_major_formatter(PercentFormatter())
ax2.set_ylim(0, 100) # TODO : tell Boat to add this line
ax.tick_params(axis="y", colors="C0")
ax2.tick_params(axis="y", colors="C1")
plt.title("Pareto chart of Absolute Error of GBDT_m")
plt.xlabel("AE_1")
ax.set_ylabel("Frequency")
ax.set_xlabel("Absolute Error")
ax2.set_ylabel("Cumulative percentage", rotation=270)
plt.grid()
plt.show()

In [ ]:
gbdt_m_result

In [ ]:
#Mon 26 Dec 65
#TODO Error dist for CV_gbdt_s,m -- done
#TODO pred/actual dist for CV_gbdt_s,m (similar to error dist)
#TODO hyperparameters - score, time, error dist -- done

# X-ต้น Y-ตาม

In [ ]:
#Mon 2 Jan 66
#TODO: plot X=actual, Y=actual/predicted for all gbdt_s and m
#TODO: fix y-axis of percentage in pareto (0, 100) ylim

## 7. Model Deployment (ONNX)

In [ ]:
from skl2onnx import __max_supported_opset__
print("Last supported opset:", __max_supported_opset__)

In [ ]:
X.info()

In [ ]:
gbdt_s = pkl.load(open("gbdt_s.pkl", "rb"))
gbdt_m = pkl.load(open("gbdt_m.pkl", "rb"))

In [ ]:
gbdt_s

In [ ]:
gbdt_m

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

In [ ]:
X.shape

In [ ]:
Xs_train[:1]

In [ ]:
Xs_train.astype(np.float32).values[:1]

In [ ]:
np.arange(20).reshape(10, 2)

In [ ]:
initial_type = [("X", Xs_train.astype(np.float32).values)]
onx = convert_sklearn(gbdt_s, initial_types=initial_type)
with open("gbdt_s.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [ ]:
import onnxruntime as rt

In [ ]:
X.info()

In [ ]:
Xs_test.info()

In [ ]:
sess = rt.InferenceSession("gbdt_s.onnx")
pred_ort = sess.run(None, {'X': Xs_test.astype(np.float32)})[0]

In [ ]:
np.random.rand(2,7).astype('float32')

In [ ]:
df